### 분류 - income

In [23]:
import pandas as pd

# 데이터 준비
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# 데이터 탐색
# train.info()
# test.info()

# print(train.isnull().sum())
# print(test.isnull().sum())

# 결측치 대체(문자형 자료) - 최빈값
m = train['workclass'].mode()[0]
train['workclass'].fillna(m, inplace=True)

m = train['native.country'].mode()[0]
train['native.country'].fillna(m, inplace=True)

m = train['occupation'].mode()[0]
train['occupation'].fillna(m, inplace=True)

test['workclass'].fillna(m, inplace=True)
test['native.country'].fillna(m, inplace=True)
test['occupation'].fillna(m, inplace=True)

# 결측치 대체 - 수치형 자료(중앙값)
mid_val = train['age'].median()
train['age'].fillna(mid_val, inplace=True)

mid_val = train['hours.per.week'].median()
train['hours.per.week'].fillna(mid_val, inplace=True)

test['age'].fillna(mid_val, inplace=True)
test['hours.per.week'].fillna(mid_val, inplace=True)

# print(train.isnull().sum())
# print(test.isnull().sum())

# 검증용 데이터 분할을 위한 작업
# 데이터 선택(X, y)
# 훈련용 칼럼(독립변수)
# train에 있던 income 칼럼을 삭제해서 y_train에 넣음
target = train.pop("income")

print(train.shape, test.shape)

# 원-핫 인코딩 - 문자(0. 1로 변환)
# train = pd.get_dummies(train)
# test = pd.get_dummies(test)

# 데이터 합치기
data = pd.concat([train, test], axis=0)
data = pd.get_dummies(data)
# 데이터 분리
train = data.iloc[:len(train)].copy()
test = data.iloc[len(test):].copy()

print(train.shape, test.shape)
print(train.info())

# cols = ["workclass", "education", "marital.status", "occupation", 
#         "relationship", "race", "sex", "native.country"]

# sklearn(사이킷런) - preprocessing, model_selection
# 인코딩(encoding) - 원핫 인코딩, 라벨 인코딩
# 문자데이터는 넣을 수 없으므로 숫자로 변환
# from sklearn.preprocessing import LabelEncoder

# for col in cols:
#   le = LabelEncoder()
#   train[col] = le.fit_transform(train[col])
#   test[col] = le.fit_transform(test[col])

# print(train.info())
# # print(test.info())
# print(y_train.value_counts())

# 검증 데이터 나누기(훈련:80, 검증: 20)
from sklearn.model_selection import train_test_split

# help(train_test_split)
X_train, X_test, y_train, y_test = train_test_split(train, 
                                                    target, 
                                                    test_size=0.2, 
                                                    random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 머신러닝 학습 및 평가
# 의사결정나무(Decision Tree) -> 랜덤 포레스트(RandomForest)
from sklearn.ensemble import RandomForestClassifier
# 모델 객체 
rf = RandomForestClassifier(random_state=42)
# 학습
rf.fit(X_train, y_train)
# 예측
pred = rf.predict_proba(X_test)  #확률
# print(pred)

# 성능 평가 - 검증용 실제값과 예측값 비교
from sklearn.metrics import roc_auc_score
# >50K인 열을 지정[: 1]
score = roc_auc_score(y_test, pred[:, 1])
print("score:", score)

# test 자료를 검증(최종)
pred2 = rf.predict_proba(test)
# print(pred2)

# 파일 생성 및 제출
submit = pd.DataFrame({
    "pred": pred2[:, 1]
})

submit.to_csv("result.csv", index=False)

pd.read_csv("result.csv")

C:\Users\admin\AppData\Local\Temp\ipykernel_11604\3713263941.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['workclass'].fillna(m, inplace=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_11604\3713263941.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

(29304, 15) (3257, 15)
(29304, 108) (29304, 108)
<class 'pandas.core.frame.DataFrame'>
Index: 29304 entries, 0 to 29303
Columns: 108 entries, id to native.country_Yugoslavia
dtypes: bool(101), float64(2), int64(5)
memory usage: 4.6 MB
None
(23443, 108) (5861, 108) (23443,) (5861,)
score: 0.9088306242549838


,pred
0,1.00
1,0.22
2,0.00
3,0.03
4,0.06
...,...
29299,0.00
29300,0.43
29301,0.12
29302,0.00
